# Calliope Urban Scale Example Model 

In [1]:
import calliope

# We increase logging verbosity
calliope.set_log_verbosity('INFO', include_solver_output=False)

In [3]:
model = calliope.examples.urban_scale()

[2023-12-09 20:58:15] INFO     Model: initialising
[2023-12-09 20:58:16] INFO     Model: preprocessing stage 1 (model_run)
[2023-12-09 20:58:16] INFO     Model: preprocessing stage 2 (model_data)
[2023-12-09 20:58:16] INFO     Model: preprocessing complete


In [4]:
# Model inputs can be viewed at `model.inputs`. 
# Variables are indexed over any combination of `techs`, `locs`, `carriers`, `costs` and `timesteps`, 
# although `techs`, `locs`, and `carriers` are often concatenated. 
# e.g. `chp`, `X1`, `heat` -> `X1::chp::heat` 
model.inputs

<xarray.Dataset>
Dimensions:                               (loc_techs: 26, loc_techs_export: 4,
                                           loc_techs_finite_resource: 9,
                                           loc_techs_supply_plus: 3,
                                           loc_techs_area: 3, timesteps: 48,
                                           costs: 1, loc_techs_om_cost: 8,
                                           loc_techs_investment_cost: 20,
                                           ...
                                           loc_tech_carriers_conversion_plus: 3,
                                           loc_carriers: 10,
                                           loc_techs_non_conversion: 23,
                                           loc_techs_conversion: 2,
                                           loc_techs_conversion_plus: 1,
                                           carriers: 3)
Coordinates: (12/20)
  * carrier_tiers                         (carrier_tiers) <U5 'in' 'out' 'out_2'
  * carriers                              (carriers) <U11 'electricity' ... '...
  * coordinates                           (coordinates) object 'x' 'y'
  * costs                                 (costs) object 'monetary'
  * loc_carriers                          (loc_carriers) object 'X3::heat' .....
  * loc_tech_carriers_conversion_plus     (loc_tech_carriers_conversion_plus) object ...
    ...                                    ...
  * loc_techs_om_cost                     (loc_techs_om_cost) object 'X1::sup...
  * loc_techs_supply_plus                 (loc_techs_supply_plus) object 'X1:...
  * loc_techs_transmission                (loc_techs_transmission) object 'N1...
  * locs                                  (locs) object 'N1' 'X1' 'X2' 'X3'
  * techs                                 (techs) object 'chp' ... 'demand_el...
  * timesteps                             (timesteps) datetime64[ns] 2005-07-...
Data variables: (12/38)
    lifetime                              (loc_techs) float64 25.0 25.0 ... nan
    energy_eff                            (loc_techs) float64 0.9037 ... nan
    energy_prod                           (loc_techs) float64 1.0 1.0 ... nan
    energy_cap_max                        (loc_techs) float64 2e+03 ... nan
    export_carrier                        (loc_techs_export) object 'electric...
    energy_con                            (loc_techs) float64 1.0 1.0 ... 1.0
    ...                                    ...
    lookup_loc_techs_conversion_plus      (carrier_tiers, loc_techs_conversion_plus) object ...
    lookup_loc_techs_export               (loc_techs_export) object 'X1::chp:...
    lookup_loc_techs_area                 (locs) object '' 'X1::pv' ... 'X3::pv'
    timestep_resolution                   (timesteps) float64 1.0 1.0 ... 1.0
    timestep_weights                      (timesteps) float64 1.0 1.0 ... 1.0
    max_demand_timesteps                  (carriers) datetime64[ns] 2005-07-0...
Attributes:
    calliope_version:    0.6.10
    applied_overrides:   
    scenario:            None
    defaults:            carrier_ratios:\ncharge_rate:\nenergy_cap_per_storag...
    allow_operate_mode:  1

In [ ]:
# Individual data variables can be accessed easily, `to_pandas()` reformats the data to look nicer
model.inputs.resource.to_pandas()

In [ ]:
# To reformat the array, deconcatenating loc_techs / loc_tech_carriers, you can use model.get_formatted_array()
# You can then apply loc/tech/carrier only operations, like summing information over locations: 
model.get_formatted_array('resource').sum('locs').to_pandas()

In [ ]:
# Solve the model. Results are loaded into `model.results`. 
# By including logging (see package importing), we can see the timing of parts of the run, as well as the solver's log
model.run()

In [ ]:
# Model results are held in the same structure as model inputs. 
# The results consist of the optimal values for all decision variables, including capacities and carrier flow
# There are also results, like system capacity factor and levelised costs, which are calculated in postprocessing
# before being added to the results Dataset

model.results

In [ ]:
# find the objective function value
model.results.objective_function_value

In [ ]:
# We can sum heat output over all locations and turn the result into a pandas DataFrame
df_heat = model.get_formatted_array('carrier_prod').loc[{'carriers':'heat'}].sum('locs').to_pandas().T

#The information about the dataframe tells us about the amount of data it holds in the index and in each column
df_heat.info()

In [ ]:
# Using .head() to see the first few rows of heat generation and demand

# Note: carrier production in heat_pipes:N1 is heat received by the heat network at any location connected to `N1`

df_heat.head()

In [ ]:
# We can plot this by using the timeseries plotting functionality.
# The top-left dropdown gives us the chance to scroll through other timeseries data too.

model.plot.timeseries()

In [ ]:
# plot.capacities gives a graphical view of the non-timeseries variables, both input and output
model.plot.capacity()

In [ ]:
# We can also examine total technology costs
# notice all the NaN values which appear when seperating loc::techs to locs and techs. 
# Any NaN value means we never considered that combination of `loc` and `tech` for the variable

costs =  model.get_formatted_array('cost').loc[{'costs': 'monetary'}].to_pandas()
costs

In [ ]:
# We can examine levelized costs for each location and technology

lcoes = model.results.systemwide_levelised_cost.loc[{'carriers': 'electricity', 'costs':'monetary'}].to_pandas()
lcoes

In [ ]:
# We can just plot this directly using calliope analysis functionality
model.plot.capacity(array='systemwide_levelised_cost')

In [ ]:
# Plotting a map of locations and transmission lines is easily possible
# In our example, the system is purely hypothetical and the resulting map
# gives us little useful information...

model.plot.transmission()

In [ ]:
# Transmission plots give us a static picture of installed capacity along links. 
# If we want timeseries data on energy flows at locations and along links
# we can use energy flow plotting. We only show the first day here, to improve loading speed.

model.plot.flows(timestep_index_subset=[0, 24])

---

See the [Calliope documentation](https://calliope.readthedocs.io/) for more details on setting up and running a Calliope model.